<a href="https://colab.research.google.com/github/adinursetya/authorobufus/blob/main/Pre_trained_Fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from gensim.corpora import WikiCorpus

dataset = WikiCorpus("/content/drive/MyDrive/Colab Notebooks/idwiki-07082019-pages-articles.xml.bz2", lemmatize=False, dictionary={})

In [ ]:
with open("wiki-id-formatted.txt", 'w', encoding="utf8") as output:
     counter = 0
     for text in dataset.get_texts():
         output.write(' '.join(text)+"\n")
         counter = counter + 1
         if counter > 10000: # Hanya mengambil 10,000 artikel pertama
             break

In [ ]:
import fasttext

model = fasttext.train_unsupervised('wiki-id-formatted.txt', model='skipgram', dim=100)

In [ ]:
model.save_model("trained_model_100_id.bin")

In [ ]:
import numpy as np
import nltk
import pandas as pd
import gensim
from nltk.tokenize import word_tokenize
from gensim.models import FastText
from nltk.tag import CRFTagger
from tqdm import tqdm

In [ ]:
model = FastText.load_fasttext_format('trained_model_100_id.bin')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/model/data.csv')
df

,id,author,content
0,1,Deti Mega Purnamasari,Jumlah pasien suspek Covid-19 hingga Senin (...
1,2,Deti Mega Purnamasari,Wakil Presiden Ma'ruf Amin meminta Dewan Syar...
2,3,Deti Mega Purnamasari,Data pemerintah menunjukkan sebanyak 39.285 s...
3,4,Deti Mega Purnamasari,Total pasien meninggal dunia akibat Covid-19...
4,5,Deti Mega Purnamasari,Sudah lebih dari tujuh bulan pandemi Covid-19...
...,...,...,...
5995,5996,Ihsanuddin,Juru bicara pemerintah untuk penanganan virus ...
5996,5997,Ihsanuddin,Pasien positif corona Covid-19 yang meninggal...
5997,5998,Ihsanuddin,Jumlah pasien positif corona Covid-19 yang d...
5998,5999,Ihsanuddin,Juru bicara penanganan virus Corona Achmad Y...


In [ ]:
real_article = df['content']
obfus_article = df1['obfsukasi']

In [ ]:
get_article_similarity(real_article, obfus_article)

0.991193581521511

In [ ]:
def clean(text):
    # Lowering letters
    text = text.lower()
    # Joining words
    text = " ".join(text.splitlines())
    return text

In [ ]:
def tokenize(text):
    word = word_tokenize(text)
    return word

In [ ]:
def pos_tag(text,ct):
    pos = ct.tag(text)
    return pos

In [ ]:
def replace(pos,filter_word,model):
    new = []
    for word in pos:
        if word[0] in filter_word:
            new.append(word[0])
        elif word[0] not in model:
            new.append(word[0])
        elif word[1] == 'VB' or word[1] == 'NN':
                word_similar = model.wv.most_similar(word[0])[0][0]
                new.append(word_similar)
        else :
            new.append(word[0])
    return new

In [ ]:
def make_document(text):
    document = ' '.join(text)
    return document

In [ ]:
def obfuskasi(data):
    ct = CRFTagger()
    ct.set_model_file('/content/drive/MyDrive/Colab Notebooks/model/all_indo_man_tag_corpus_model.crf.tagger')
    model = FastText.load_fasttext_format('trained_model_100_id.bin')

    filter_word = ['senin','selasa','rabu','kamis','jumat','sabtu','minggu','januari','februari',
              'maret','april','mei','juni','juli','agustus','september','oktober','november','desember',
              'pagi','siang','sore','petang','malam','jam','menit','detik',]
    new = []
    for i in tqdm(data):
        text = clean(i)
        word = tokenize(text)
        word = pos_tag(word,ct)
        document = replace(word,filter_word,model)
        new_document = make_document(document)
        new.append(new_document)
    return new

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/model/data.csv')
df

In [ ]:
data = df['content'].values
data

In [ ]:
new = obfuskasi(data)
new

In [ ]:
Id = [i for i in range(len(new))]

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/output3.csv')
df1

In [ ]:
df_new = pd.DataFrame(Id,columns=['id'])
df_new['author'] = df['author']
df_new['obfsukasi'] = new
df_new.to_csv('output3.csv', index = False)

In [ ]:
def get_article_similarity(real_article, obfus_article):
    word_similarities = []

    for i in range(len(real_article)):
        if len(real_article[i]) > 1 and len(obfus_article[i]) > 1:
            word1 = real_article[i].lower()
            word2 = obfus_article[i].lower()
            similarity = 1

            if word1 != word2:
                    similarity = model.wv.similarity(word1, word2)

            word_similarities.append(similarity)

    average_similarity = sum(word_similarities) / len(word_similarities)

    return average_similarity